<a href="https://colab.research.google.com/github/Meta-Sean/sqzme/blob/main/VIX_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [ ]:
# list of products
# https://www.cboe.com/us/futures/market_statistics/historical_data/
# VX is VIX, VXM is VX Mini

In [ ]:
product = 'VXM' 

In [ ]:
url = f"https://www.cboe.com/us/futures/market_statistics/historical_data/products/{product}"
page = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(page.content, 'html.parser')
links = [link.get('href') for link in soup.find_all('a')]
cfe_links = [link for link in links if f'products/csv/{product}/' in link]

In [ ]:
cfe_futures = pd.DataFrame(columns = ['Trade Date', 'Futures', 'Open', 'High', 'Low', 'Close', 'Settle',
       'Change', 'Total Volume', 'EFP', 'Open Interest', 'Expiration Date', 'Product'])

In [ ]:
for link in cfe_links:
    expiration_date = list(filter(None, link.split('/')))[-1]

    if expiration_date in cfe_futures['Expiration Date'].values:
        print(f"Skipping {expiration_date} because it already exists")
        continue

    print(f"Downloading {expiration_date}")

    csv_url = f"""https://www.cboe.com/us/futures/market_statistics/historical_data/{link}"""
    df = pd.read_csv(csv_url)
    df['Expiration Date'] = expiration_date
    df['Product'] = product

    cfe_futures = pd.concat([cfe_futures, df])

In [ ]:
cfe_futures.to_pickle(f'{product}_futures.pkl')